In [15]:
from tensorflow.keras import models, layers, activations, initializers

model = models.Sequential([
    layers.Input(shape=(28,28,3)), # 28x28x1

    layers.Resizing(28,28),
    layers.Rescaling(1./255),
    layers.RandomRotation((-0.2, 0.2)),

    layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(), # 7840
    layers.Dense(64, activation=activations.sigmoid, kernel_initializer=initializers.RandomNormal()),
    layers.Dropout(0.2),
    layers.Dense(64, activation=activations.sigmoid, kernel_initializer=initializers.RandomNormal()),

    layers.Dense(24, activation=activations.softmax, kernel_initializer=initializers.RandomNormal()) # 24
])

from tensorflow.keras import optimizers, losses, metrics

lr = 0.001

model.compile(
    optimizer = optimizers.Adam(
        learning_rate = lr    
    ),
    loss = losses.SparseCategoricalCrossentropy(),
    metrics = [ metrics.sparse_categorical_accuracy ]
)

from tensorflow.keras import utils

path = './Data'
batch_size = 64

train = utils.image_dataset_from_directory(
    directory=path + '/Train',
    shuffle = True,
    seed = 1,
    image_size = (28,28),
    batch_size = batch_size
)

test = utils.image_dataset_from_directory(
    directory=path + '/Test',
    shuffle = True,
    seed = 1,
    image_size = (28,28),
    batch_size = batch_size
)

from tensorflow.keras import callbacks

model_path = "./model.keras"

patience = 3
epochs = 100

model.fit(
    train,
    validation_data = test,
    epochs = epochs,
    verbose = True,
    
    callbacks = [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks .ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

Found 27455 files belonging to 24 classes.
Found 7172 files belonging to 24 classes.
Epoch 1/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 12s 25ms/step - loss: 2.9302 - sparse_categorical_accuracy: 0.1225 - val_loss: 2.3242 - val_sparse_categorical_accuracy: 0.2471
Epoch 2/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - loss: 1.9483 - sparse_categorical_accuracy: 0.3328 - val_loss: 1.7035 - val_sparse_categorical_accuracy: 0.4024
Epoch 3/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - loss: 1.4646 - sparse_categorical_accuracy: 0.4893 - val_loss: 1.3525 - val_sparse_categorical_accuracy: 0.5318
Epoch 4/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - loss: 1.0959 - sparse_categorical_accuracy: 0.6201 - val_loss: 1.0604 - val_sparse_categorical_accuracy: 0.6104
Epoch 5/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step - loss: 0.8073 - sparse_categorical_accuracy: 0.7222 - val_loss: 0.7592 - val_sparse_categorical_accuracy: 0.7345
Epoch 6/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 12s 27ms/step - loss: 0.5885 